In [1]:
!pip install groq python-dotenv numpy tqdm datasets

^C
Traceback (most recent call last):
  File "/Users/jimin/anaconda3/envs/cot/bin/pip", line 11, in <module>
    sys.exit(main())
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 177, in handle_pip_version_check
    session = self._build_session(
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 122, in _build_session
    session = PipSession(
  File "/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/pip/_i

In [6]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

/Users/jimin/anaconda3/envs/cot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192",
        temperature: float = 0.3
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=temperature, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [21]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you may have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're trying to understand? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [22]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [25]:
from collections import Counter

def run_benchmark_test(
        dataset,
        prompt,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False,
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        if isinstance(prompt, list):
            response_list = []
            for p in prompt:
                response = generate_response_using_Llama(
                    prompt=p.format(question=question),
                    model=model
                )
                response_list.append(response)
        else:
            response = generate_response_using_Llama(
                prompt=prompt.format(question=question),
                model=model
            )
            response_list = [response]

        predicted_answer = None  # 초기화

        if response_list:
            if VERBOSE:
                print("="*50)
                for idx, res in enumerate(response_list):
                    print(f"[Response {idx+1}]\n{res}\n")
                print("="*50)

            # 각 응답에서 최종 숫자 추출
            extracted_answers = [extract_final_answer(r) for r in response_list]
            try:
                extracted_answers = [
                    float(ans.replace(",", "")) for ans in extracted_answers if ans is not None
                ]
            except Exception as e:
                if VERBOSE:
                    print(f"⚠️ Error during prediction: {e}")
                predicted_answer = None

            if len(extracted_answers) == 0:
                response = generate_response_using_Llama(
                    prompt=prompt.format(question=question),
                    model=model,
                    temperature=0.2
                )
                response_list = [response]
                extracted_answers = [extract_final_answer(r) for r in response_list]
                try:
                    extracted_answers = [
                        float(ans.replace(",", "")) for ans in extracted_answers if ans is not None
                    ]
                except Exception as e:
                    if VERBOSE:
                        print(f"⚠️ Error during prediction: {e}")
                    predicted_answer = None

            
            if extracted_answers:
                # 🎯 다수결 적용
                most_common = Counter(extracted_answers).most_common(1)[0][0]
                predicted_answer = most_common
            else:
                predicted_answer = None

            is_correct = abs(predicted_answer - correct_answer) < 1e-5 if predicted_answer is not None else False

            if is_correct:
                correct += 1
            total += 1

            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response_list if isinstance(prompt, list) else response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct / total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct / total if total > 0 else 0

In [7]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [7]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  1.82it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:05<00:00,  1.84it/s]

Progress: [10/10]
Current Acc.: [20.00%]


In [11]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

### 0 shot direct prompting
PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")

### 3 shot direct prompting
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")

### 5 shot direct prompting
PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")



 10%|█         | 5/50 [00:02<00:22,  2.02it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:04<00:19,  2.07it/s]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:07<00:20,  1.72it/s]

...

Progress: [40/50]
Current Acc.: [22.50%]


 90%|█████████ | 45/50 [02:02<00:13,  2.68s/it]

Progress: [45/50]
Current Acc.: [20.00%]


100%|██████████| 50/50 [02:15<00:00,  2.71s/it]

Progress: [50/50]
Current Acc.: [20.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [13]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nGenerate the answer for the given mathematical question with step-by-step rationale toward the answer."

    for i in range(num_examples):
        cur_question = gsm8k_train['question'][sampled_indices[i]]
        cur_answer = gsm8k_train['answer'][sampled_indices[i]]
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Output:{cur_answer}\n"

    prompt += f"\n\n[Example {num_examples+1}]\n"
    prompt += "Question:\n{question}\n"
    prompt += "#### "


    return prompt

In [14]:
print(construct_CoT_prompt(5))

Instruction:
Generate the answer for the given mathematical question with step-by-step rationale toward the answer.
[Example 1]
Question:
Lena played video games for 3.5 hours last weekend. Her brother played 17 minutes more than she did. How many minutes together did Lena and her brother play video games last weekend?
Output:Lena = 3.5 hours = 210 minutes
Brother = 210 + 17 = <<210+17=227>>227 minutes
210 + 227 = <<210+227=437>>437 minutes
Together, Lena and her brother played 437 minutes of video games.
#### 437

[Example 2]
Question:
There are 3 rows of people relaxing at the beach. The first row is made up of 24 people until 3 people get up to wade in the water. 5 people from the second row, which originally held 20 people, go to join them. The third row is made up of 18 people. How many people are left relaxing on the beach?
Output:In the first row, there are now 24 – 3 = <<24-3=21>>21 people.
In the second row, there are now 20 – 5 = <<20-5=15>>15 people.
So in total there are 21

In [15]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

PROMPT = construct_CoT_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")



 10%|█         | 5/50 [00:12<01:46,  2.38s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:29<02:08,  3.22s/it]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:36<00:53,  1.52s/it]

...

Progress: [40/50]
Current Acc.: [57.50%]


 90%|█████████ | 45/50 [01:47<00:16,  3.20s/it]

Progress: [45/50]
Current Acc.: [57.78%]


100%|██████████| 50/50 [02:05<00:00,  2.50s/it]

Progress: [50/50]
Current Acc.: [58.00%]


In [ ]:
### 3 shot CoT prompting
PROMPT = construct_CoT_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_3.txt")

 10%|█         | 5/50 [00:04<00:43,  1.04it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:50<05:19,  7.99s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [01:35<05:18,  9.11s/it]

...

Progress: [40/50]
Current Acc.: [62.50%]


 90%|█████████ | 45/50 [05:53<00:43,  8.61s/it]

Progress: [45/50]
Current Acc.: [66.67%]


100%|██████████| 50/50 [06:38<00:00,  7.98s/it]

Progress: [50/50]
Current Acc.: [68.00%]


In [ ]:
### 5 shot CoT prompting
PROMPT = construct_CoT_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=True,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_5.txt")

### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [8]:
def construct_my_prompt(num_examples: int = 3) -> str:
    """
    Construct a few-shot prompt enforcing step-by-step breakdown and answer verification.
    """

    prompt = (
        "You are a meticulous math tutor. Solve each problem by explicitly breaking it down into smaller subproblems.\n"
        "Then verify your answer by recomputing it at the end. Always end your response with '#### [final answer]'.\n\n"
    )

    sampled_indices = random.sample(range(len(gsm8k_train['question'])), num_examples)

    for i, idx in enumerate(sampled_indices):
        question = gsm8k_train['question'][idx].strip()
        answer = gsm8k_train['answer'][idx].strip()
        prompt += f"[Example {i+1}]\n"
        prompt += f"Question:\n{question}\n"
        prompt += f"Answer:\nLet's break this into parts step-by-step.\n"
        prompt += f"{answer}\n"
        prompt += "Let's verify the final answer by recomputing it...\n"
        prompt += f"{answer.split('####')[0].strip()}\n{answer.strip()}\n\n"

    prompt += f"[Example {num_examples+1}]\n"
    prompt += "Question:\n{question}\n"
    prompt += "Answer:\nLet's break this into parts step-by-step.\n"

    return prompt

In [ ]:
### 0 shot direct prompting

PROMPT = construct_my_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt = PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50,
)
save_final_result(results, accuracy, "My_prompting_0.txt")

In [ ]:
### 3 shot direct prompting

PROMPT = construct_my_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt = PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50,
)
save_final_result(results, accuracy, "My_prompting_3.txt")

In [11]:
### 5 shot direct prompting

PROMPT = construct_my_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt = PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50,
)
save_final_result(results, accuracy, "My_prompting_5.txt")

 10%|█         | 5/50 [01:42<17:35, 23.46s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [03:55<17:19, 26.00s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [06:15<15:39, 26.83s/it]

...

Progress: [40/50]
Current Acc.: [70.00%]


 90%|█████████ | 45/50 [20:52<02:47, 33.59s/it]

Progress: [45/50]
Current Acc.: [68.89%]


100%|██████████| 50/50 [22:58<00:00, 27.57s/it]

Progress: [50/50]
Current Acc.: [70.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!